### Libraries

In [1]:
from datetime import datetime
from typing import Annotated, TypedDict
from pydantic import BaseModel, Field

from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities import PythonREPL

from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

### Tool - image generation

In [3]:
client = OpenAI()


# Define schema for generating image using DALL-E
class GenImageSchema(BaseModel):
    prompt: str = Field(description="The prompt for image generation")


@tool(args_schema=GenImageSchema)
def generate_image(prompt: str) -> str:
    """
    Generate an image using DALL-E based on the given prompt.
    """
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )
    return response.data[0].url

### Tool - Code 

In [4]:
repl = PythonREPL()


@tool
def python_repl(
    code: Annotated[str, "The python code to excute to generate your chart."],
):
    """
    Use this to execute python code.
    This is visible to the user. Please use ploly to visualize.
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str

### Agents

In [6]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

system_prompt = f"""
Today is {datetime.now().strftime("%Y-%m-%d")}
You are a helpful AI Assistant that can use web search tool(tavily ai api), image generation tool(DallE API) and code execution tool(Python REPL).
You should always answer in same language as user's ask.
When user ask about the information that you can't answer, you can call the search tool.
When user ask about generating image, you can call the generate_image tool.
When user ask about Data analysis, data visualization or code execution image, you can call the python repl tool.
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

search = TavilySearchResults(max_results=5)

tools = [search, generate_image, python_repl]
llm_with_tools = llm.bind_tools(tools)

chain = prompt | llm_with_tools

In [7]:
result = chain.invoke("대한민국 GDP 추이를 그래프로 그려줘")
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_LZakCy4UyoRjj8wd2m7zif01', 'function': {'arguments': '{"query":"대한민국 GDP 추이"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 283, 'total_tokens': 305, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f7eb51d9-2346-4275-a0fd-5ea5acef2fff-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '대한민국 GDP 추이'}, 'id': 'call_LZakCy4UyoRjj8wd2m7zif01', 'type': 'tool_call'}] usage_metadata={'input_tokens': 283, 'output_tokens': 22, 'total_tokens': 305, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {

### Building Graph

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


def chatbot(state: State):
    return {"messages": [chain.invoke(state["messages"])]}


tool_node = ToolNode(tools=tools)

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph = graph_builder.compile()

In [ ]:
# The config is the **second positional argument** to stream() or invoke()!
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", input("User: "))]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()